Start OSM project!

In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "austin_texas.osm"  # Replace this with your osm file
SAMPLE_FILE = "austin_sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            print(ET.tostring(element, encoding='iso8859-1'))
            output.write(ET.tostring(element, encoding='iso8859-1'))

    output.write('</osm>')

<Element 'node' at 0x0000000071C5D1D8>
b'<?xml version=\'1.0\' encoding=\'iso8859-1\'?>\n<node changeset="8497118" id="26546004" lat="30.4695355" lon="-97.7972587" timestamp="2011-06-20T18:36:15Z" uid="388279" user="Tylan" version="15" />\n\t'


TypeError: write() argument must be str, not bytes

In [23]:
# -*- coding: utf-8 -*-

from collections import defaultdict, Counter
import csv
import pprint
import re
import sqlite3
import xml.etree.cElementTree as ET

OSM_FILE = open('austin_texas.osm', 'r', encoding="utf8")

In [24]:
# -*- coding: utf-8 -*-
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
tags = {}
child_tags = {}

for element in get_element(OSM_FILE):
    if element.tag in tags.keys():
        tags[element.tag] += 1
    else:
        tags[element.tag] = 1
    for child in element:
        if child.tag in child_tags.keys():
            child_tags[child.tag] += 1
        else:
            child_tags[child.tag] = 1
    
print("\nFor the {}".format(OSM_FILE))
print(tags)
print(child_tags)


For the <_io.TextIOWrapper name='austin_texas.osm' mode='r' encoding='utf8'>
{'relation': 2400, 'node': 6389941, 'way': 669817}
{'tag': 2388234, 'member': 20388, 'nd': 7024440}


In [16]:
# Set up SQL database for austin data and initialize cursor obj
db_conn = sqlite3.connect('austin_osm.db')
c = db_conn.cursor()